In [ ]:
!pip install keras.metrics --quiet
!pip install transformers --quiet
!pip install langdetect --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 5.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done


In [ ]:
import numpy as np
import pandas as pd
import os
import string
from string import digits
import matplotlib.pyplot as plt
%matplotlib inline
import re
import pickle
import seaborn as sns
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from keras.layers import Input, LSTM, Embedding, Dense, Dropout, RepeatVector, Dot, Concatenate, Bidirectional, Activation
from keras.models import Model

In [ ]:
en_file = r'/content/drive/MyDrive/Honours-DL/Translation/train.en'
mr_file = r'/content/drive/MyDrive/Honours-DL/Translation/train.mr'

In [ ]:
with open(en_file, 'r', encoding='utf-8') as file:
    en_sentences = [line.strip() for line in file.readlines()[:20000]]
with open(mr_file, 'r', encoding='utf-8') as file:
    mr_sentences = [line.strip() for line in file.readlines()[:20000]]
en_sentences = [sentence.rstrip('\n') for sentence in en_sentences]
mr_sentences = [sentence.rstrip('\n') for sentence in mr_sentences]
print("Eng : Total", len(en_sentences))
print("Mar : Total ", len(mr_sentences))

Eng : Total 20000
Mar : Total  20000


In [ ]:
# with open(en_file, 'r') as file:
#     en_sentences = file.readlines()
# with open(mr_file, 'r') as file:
#     mr_sentences = file.readlines()


# en_sentences = [sentence.rstrip('\n') for sentence in en_sentences]
# mr_sentences = [sentence.rstrip('\n') for sentence in mr_sentences]

In [ ]:
lines =  pd.DataFrame({"English":en_sentences, "Marathi":mr_sentences })
lines.shape

(20000, 2)

In [ ]:
lines.head(5)

,English,Marathi
0,"Today, ASEAN and India enjoy multi-faceted coo...","आज आसियान आणि भारत, आसियानचे राजनीतिक- सुरक्षा..."
1,"For us in India, ASEANs continuance in this ro...",आसियान राष्ट्र समुहातील देशांशी भारताचे उत्तम ...
2,We need to put faith in his reminders.,[ ७ पानांवरील चित्र]
3,Presently elections are being held in Gujarat.,"दरम्यान, गुजरात निवडणूकीचा प्रचार आता शिगेला प..."
4,This will aim to pool research and technologic...,यामुळे सौर ऊर्जा क्षेत्रातल्या संशोधनाला आणि त...


In [ ]:
lines=lines.sample(n=350000, random_state=42)
print(lines.shape)

In [ ]:
lines['English']=lines['English'].apply(lambda x: x.lower())
lines['Marathi']=lines['Marathi'].apply(lambda x: x.lower())

In [ ]:
lines['English']=lines['English'].apply(lambda x: re.sub("'", '', x))
lines['Marathi']=lines['Marathi'].apply(lambda x: re.sub("'", '', x))

In [ ]:
exclude = set(string.punctuation) # Set of all special characters
print(exclude)
# Remove all the special characters
lines['English']=lines['English'].apply(lambda x: ''.join(ch for ch in x if ch not in exclude))
lines['Marathi']=lines['Marathi'].apply(lambda x: ''.join(ch for ch in x if ch not in exclude))

{'%', '>', '_', '$', ']', '.', '}', '?', '*', '"', '&', ':', '<', ';', '~', '+', '!', '#', '(', ',', '\\', '^', '/', ')', '|', '-', '=', '`', '@', '[', "'", '{'}


In [ ]:
# Remove all numbers from text
remove_digits = str.maketrans('', '', digits)
lines['English']=lines['English'].apply(lambda x: x.translate(remove_digits))
lines['Marathi']=lines['Marathi'].apply(lambda x: x.translate(remove_digits))

lines['Marathi'] = lines['Marathi'].apply(lambda x: re.sub("[२३०८१५७९४६]", "", x))

# Remove extra spaces
lines['English']=lines['English'].apply(lambda x: x.strip())
lines['Marathi']=lines['Marathi'].apply(lambda x: x.strip())
lines['English']=lines['English'].apply(lambda x: re.sub(" +", " ", x))
lines['Marathi']=lines['Marathi'].apply(lambda x: re.sub(" +", " ", x))

In [ ]:
lines['English'] = lines['English'].apply(lambda x : 'START_ '+ x + ' _END')

In [ ]:
### Get English and Marathi Vocabulary
all_eng_words=set()
for eng in lines['English']:
    for word in eng.split():
        if word not in all_eng_words:
            all_eng_words.add(word)

all_marathi_words=set()
for hin in lines['Marathi']:
    for word in hin.split():
        if word not in all_marathi_words:
            all_marathi_words.add(word)

In [ ]:
print(len(all_eng_words), len(all_marathi_words))

19433 38933


In [ ]:
from transformers import BertTokenizer
tokenizer = BertTokenizer.from_pretrained("google-bert/bert-base-multilingual-cased")
# यामुळे सौर ऊर्जा क्षेत्रातल्या संशोधनाला
# vocab = tokenizer.get_vocab()
tokenizer.tokenize("यामुळे सौर ऊर्जा क्षेत्रातल्या संशोधनाला !")
# print(vocab)

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/872k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.72M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/625 [00:00<?, ?B/s]

['या',
 '##मळ',
 'स',
 '##ौर',
 'ऊ',
 '##रजा',
 'कषतर',
 '##ात',
 '##लया',
 'स',
 '##श',
 '##ोध',
 '##ना',
 '##ला',
 '!']

In [ ]:
lines['length_eng_sentence']=lines['English'].apply(lambda x:len(x.split(" ")))
lines['length_mar_sentence']=lines['Marathi'].apply(lambda x:len(x.split(" ")))

In [ ]:
lines.head()

,English,Marathi,length_eng_sentence,length_mar_sentence
0,START_ today asean and india enjoy multifacete...,आज आसियान आणि भारत आसियानचे राजनीतिक सुरक्षा आ...,39,52
1,START_ for us in india aseans continuance in t...,आसियान राष्ट्र समुहातील देशांशी भारताचे उत्तम ...,29,20
2,START_ we need to put faith in his reminders _END,पानांवरील चित्र,10,2
3,START_ presently elections are being held in g...,दरम्यान गुजरात निवडणूकीचा प्रचार आता शिगेला पो...,9,8
4,START_ this will aim to pool research and tech...,यामुळे सौर ऊर्जा क्षेत्रातल्या संशोधनाला आणि त...,33,22


In [ ]:
lines[lines['length_mar_sentence']>20].shape

(1209, 4)

In [ ]:
lines=lines[lines['length_eng_sentence']<=20]
lines=lines[lines['length_mar_sentence']<=20]

In [ ]:
print(lines.shape)

(16995, 4)


In [ ]:
print("maximum length of Marathi Sentence ",max(lines['length_mar_sentence']))
print("maximum length of English Sentence ",max(lines['length_eng_sentence']))

maximum length of Marathi Sentence  20
maximum length of English Sentence  20


In [ ]:
max_length_src=max(lines['length_mar_sentence'])
max_length_tar=max(lines['length_eng_sentence'])

Load Vocab

In [ ]:
with open(r'/content/drive/MyDrive/Honours-DL/Translation/Vocab/marathi.pkl', "rb") as f:
    marathi_vocab = pickle.load(f)
with open(r'/content/drive/MyDrive/Honours-DL/Translation/Vocab/english.pkl', "rb") as f:
    english_vocab = pickle.load(f)

In [ ]:
input_words = sorted(list(all_marathi_words))
target_words = sorted(list(all_eng_words))
num_encoder_tokens = len(all_marathi_words)
num_decoder_tokens = len(all_eng_words)
print(num_encoder_tokens, num_decoder_tokens)


38933 19433


In [ ]:
num_decoder_tokens += 1 #for zero padding

In [ ]:
input_token_index = dict([(word, i+1) for i, word in enumerate(input_words)])
target_token_index = dict([(word, i+1) for i, word in enumerate(target_words)])
print(list(target_token_index.items())[50:80])

[('abe', 51), ('abel', 52), ('aberdeen', 53), ('abetment', 54), ('abetting', 55), ('abhay', 56), ('abhijeet', 57), ('abhijit', 58), ('abhilash', 59), ('abhilasha', 60), ('abhimanyu', 61), ('abhinandan', 62), ('abhinash', 63), ('abhinav', 64), ('abhinay', 65), ('abhishek', 66), ('abhiyan', 67), ('abhorreth', 68), ('abhyankar', 69), ('abiathar', 70), ('abide', 71), ('abiding', 72), ('abigail', 73), ('abihu', 74), ('abilities', 75), ('ability', 76), ('abimelech', 77), ('ablaze', 78), ('able', 79), ('abn', 80)]


In [ ]:
reverse_input_char_index = dict((i, word) for word, i in input_token_index.items())
reverse_target_char_index = dict((i, word) for word, i in target_token_index.items())
print(reverse_target_char_index)

{1: 'START_', 2: '_END', 3: 'a', 4: 'aadhaar', 5: 'aadhar', 6: 'aaditya', 7: 'aadmi', 8: 'aai', 9: 'aaj', 10: 'aakash', 11: 'aaliyah', 12: 'aam', 13: 'aamir', 14: 'aamirs', 15: 'aanchal', 16: 'aap', 17: 'aaps', 18: 'aaradhya', 19: 'aarey', 20: 'aarogya', 21: 'aaron', 22: 'aarti', 23: 'aaryan', 24: 'aashada', 25: 'aastha', 26: 'aatamnirbhar', 27: 'aayog', 28: 'ab', 29: 'aback', 30: 'abandon', 31: 'abandoned', 32: 'abarim', 33: 'abate', 34: 'abatement', 35: 'abba', 36: 'abbas', 37: 'abbasi', 38: 'abbotts', 39: 'abbreviated', 40: 'abcd', 41: 'abdominal', 42: 'abducting', 43: 'abduction', 44: 'abductioncumrobbery', 45: 'abductions', 46: 'abdul', 47: 'abdulaziz', 48: 'abdullah', 49: 'abdullahs', 50: 'abdur', 51: 'abe', 52: 'abel', 53: 'aberdeen', 54: 'abetment', 55: 'abetting', 56: 'abhay', 57: 'abhijeet', 58: 'abhijit', 59: 'abhilash', 60: 'abhilasha', 61: 'abhimanyu', 62: 'abhinandan', 63: 'abhinash', 64: 'abhinav', 65: 'abhinay', 66: 'abhishek', 67: 'abhiyan', 68: 'abhorreth', 69: 'abhya

My Tokeinzation Logic

In [ ]:
### Get English and Marathi Vocabulary
all_eng_words=set()
for eng in lines['English']:
    for word in tokenizer.tokenize(eng):
        if word not in all_eng_words:
            all_eng_words.add(word)

all_marathi_words=set()
for hin in lines['Marathi']:
    for word in tokenizer.tokenize(hin):
        if word not in all_marathi_words:
            all_marathi_words.add(word)

NameError: name 'tokenizer' is not defined

In [ ]:
print(len(all_eng_words), len(all_marathi_words))
print(sorted(list(all_eng_words)[30000:30050]))

19433 38933
[]


In [ ]:
e = sorted(list(all_eng_words))
m = sorted(list(all_marathi_words))
with open(r'/content/drive/MyDrive/Honours-DL/Translation/Vocab/marathi.pkl', "wb") as f:
    pickle.dump(m, f)
with open(r'/content/drive/MyDrive/Honours-DL/Translation/Vocab/english.pkl', "wb") as f:
    pickle.dump(e, f)

In [ ]:
lines = shuffle(lines)
lines.head(10)

,English,Marathi,length_eng_sentence,length_mar_sentence
19812,START_ with billion people china is worlds mos...,सध्या चीन हा जगातील सर्वाधिक लोकसंख्येचा देश अ...,11,12
6891,START_ ram temple back in bjp manifesto _END,भाजपच्या जाहीरनाम्यात राम मंदिराचा पुनरूच्चार,8,5
17438,START_ the question was _END,’असा प्रश्न पडला असता,5,4
3217,START_ it is not clear yet why he committed su...,नेमकी त्यांनी आत्महत्या कोणत्या कारणावरून केली...,11,10
11992,START_ each modality has its advantages and di...,प्रत्येक प्रदर्शन प्रकार त्याच्या स्वत च्या फा...,9,10
13627,START_ he is being taken to the cbi headquarte...,सध्या त्याला कडेकोट बंदोबस्तात दिल्लीतील सीबीआ...,12,10
5796,START_ students cannot read _END,त्यामुळे विद्यार्थी अभ्यास करीत नसत,5,5
18301,START_ the government is trying to get these w...,हे सरकार कामगारांना वेठबिगार बनवण्याच्या प्रयत...,11,7
11339,START_ when some work is actually commenced th...,जेव्हा प्रत्यक्ष कामाची सुरुवात होते तेव्हाच न...,13,9
354,START_ no one is paying attention _END,कुणाचे कुणाकडे लक्ष नसते,7,4


In [ ]:
X, y = lines['Marathi'], lines['English']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.1,random_state=42)
X_train.shape, X_test.shape

((15295,), (1700,))

In [ ]:
def generate_batch(X = X_train, y = y_train, batch_size = 128):
    ''' Generate a batch of data '''
    while True:
        for j in range(0, len(X), batch_size):
            encoder_input_data = np.zeros((batch_size, max_length_src),dtype='float32')
            decoder_input_data = np.zeros((batch_size, max_length_tar),dtype='float32')
            decoder_target_data = np.zeros((batch_size, max_length_tar, num_decoder_tokens),dtype='float32')
            for i, (input_text, target_text) in enumerate(zip(X[j:j+batch_size], y[j:j+batch_size])):
                for t, word in enumerate(input_text.split()):
                    encoder_input_data[i, t] = input_token_index[word] # encoder input seq
                for t, word in enumerate(target_text.split()):
                    if t<len(target_text.split())-1:
                        decoder_input_data[i, t] = target_token_index[word] # decoder input seq
                    if t>0:
                        # decoder target sequence (one hot encoded)
                        # does not include the START_ token
                        # Offset by one timestep
                        decoder_target_data[i, t - 1, target_token_index[word]] = 1.
            yield([encoder_input_data, decoder_input_data], decoder_target_data)

In [ ]:
def generate_batch(X=X_train, y=y_train, batch_size=128):
    ''' Generate a batch of data '''
    while True:
        for j in range(0, len(X), batch_size):
            encoder_input_data = np.zeros((batch_size, max_length_src), dtype='float32')
            decoder_input_data = np.zeros((batch_size, max_length_tar), dtype='float32')
            decoder_target_data = np.zeros((batch_size, max_length_tar, num_decoder_tokens), dtype='float32')
            for i, (input_text, target_text) in enumerate(zip(X[j:j+batch_size], y[j:j+batch_size])):
                # Tokenize input and target sentences using your tokenizer
                input_tokens = tokenizer.tokenize(input_text)
                target_tokens = tokenizer.tokenize(target_text)

                for t, word in enumerate(input_tokens):
                    if t < max_length_src:
                        encoder_input_data[i, t] = input_token_index[word]  # encoder input seq
                for t, word in enumerate(target_tokens):
                    if t < len(target_tokens) - 1 and t < max_length_tar:
                        decoder_input_data[i, t] = target_token_index[word]  # decoder input seq
                    if t > 0 and t < max_length_tar:
                        # decoder target sequence (one hot encoded)
                        # does not include the START_ token
                        # Offset by one timestep
                        decoder_target_data[i, t - 1, target_token_index[word]] = 1.
            yield ([encoder_input_data, decoder_input_data], decoder_target_data)


In [ ]:
latent_dim=128
dropout_rate = 0.2

In [ ]:
# Encoder
encoder_inputs = Input(shape=(None,))
enc_emb =  Embedding(num_encoder_tokens, latent_dim, mask_zero = True)(encoder_inputs)
encoder_lstm = LSTM(latent_dim, return_state=True)
encoder_outputs, state_h, state_c = encoder_lstm(enc_emb)
# We discard `encoder_outputs` and only keep the states.
encoder_states = [state_h, state_c]

# Set up the decoder, using `encoder_states` as initial state.
decoder_inputs = Input(shape=(None,))
dec_emb_layer = Embedding(num_decoder_tokens, latent_dim*2, mask_zero = True)
dec_emb = dec_emb_layer(decoder_inputs)
# We set up our decoder to return full output sequences,
# and to return internal states as well. We don't use the
# return states in the training model, but we will use them in inference.
decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(dec_emb,
                                     initial_state=encoder_states)
decoder_dense = Dense(num_decoder_tokens, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

# Define the model that will turn
# `encoder_input_data` & `decoder_input_data` into `decoder_target_data`
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

[<KerasTensor: shape=(None, 128) dtype=float32 (created by layer 'lstm_7')>, <KerasTensor: shape=(None, 128) dtype=float32 (created by layer 'lstm_7')>]


In [ ]:
from keras.layers import Input, LSTM, Embedding, Dense, Bidirectional, concatenate, Attention
from keras.models import Model

# Encoder
encoder_inputs = Input(shape=(None,))
enc_emb =  Embedding(num_encoder_tokens, latent_dim, mask_zero=True)(encoder_inputs)
encoder_lstm = Bidirectional(LSTM(latent_dim, return_sequences=True, return_state=True))
encoder_outputs, forward_h, forward_c, backward_h, backward_c = encoder_lstm(enc_emb)
state_h = concatenate([forward_h, backward_h])
state_c = concatenate([forward_c, backward_c])
encoder_states = [state_h, state_c]

print(encoder_inputs)
print(encoder_states)

# # Decoder
decoder_inputs = Input(shape=(None,))
dec_emb_layer = Embedding(num_decoder_tokens, latent_dim, mask_zero=True)
dec_emb = dec_emb_layer(decoder_inputs)
decoder_lstm = LSTM(latent_dim*2, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(dec_emb, initial_state=encoder_states)

print(decoder_inputs)
# decoder_concat_input = Concatenate(axis=-1)([decoder_outputs, attention])
decoder_dense = Dense(num_decoder_tokens, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

# # Model
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

KerasTensor(type_spec=TensorSpec(shape=(None, None), dtype=tf.float32, name='input_19'), name='input_19', description="created by layer 'input_19'")
[<KerasTensor: shape=(None, 256) dtype=float32 (created by layer 'concatenate_6')>, <KerasTensor: shape=(None, 256) dtype=float32 (created by layer 'concatenate_7')>]
KerasTensor(type_spec=TensorSpec(shape=(None, None), dtype=tf.float32, name='input_20'), name='input_20', description="created by layer 'input_20'")


My Model with Different input tokens

In [ ]:
# Encoder
encoder_inputs = Input(shape=(None,))
enc_emb = Embedding(num_encoder_tokens, latent_dim, mask_zero=True)(encoder_inputs)
encoder_lstm = LSTM(latent_dim, return_state=True)
encoder_outputs, state_h, state_c = encoder_lstm(enc_emb)
encoder_states = [state_h, state_c]

# Decoder
decoder_inputs = Input(shape=(None,))
dec_emb_layer = Embedding(num_decoder_tokens, latent_dim, mask_zero=True)
dec_emb = dec_emb_layer(decoder_inputs)

decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(dec_emb, initial_state=encoder_states)

decoder_dense = Dense(num_decoder_tokens, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

In [ ]:
# model.compile(optimizer='rmsprop', loss='categorical_crossentropy')
from keras.metrics import Accuracy
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
model.summary()

Model: "model_13"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_19 (InputLayer)       [(None, None)]               0         []                            
                                                                                                  
 embedding_6 (Embedding)     (None, None, 128)            4983424   ['input_19[0][0]']            
                                                                                                  
 input_20 (InputLayer)       [(None, None)]               0         []                            
                                                                                                  
 bidirectional_3 (Bidirecti  [(None, None, 256),          263168    ['embedding_6[0][0]']         
 onal)                        (None, 128),                                                 

In [ ]:
train_samples = len(X_train)
val_samples = len(X_test)
batch_size = 64
epochs = 10

In [ ]:
import tensorflow as tf

checkpoint_filepath = r'/content/drive/MyDrive/Honours-DL/Translation/temp2.h5'
model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_filepath,
    save_best_only=True,
    monitor='loss',
    mode='min',
    verbose=1
)


In [ ]:
# history = model.fit_generator(generator = generate_batch(X_train, y_train, batch_size = batch_size),
#                     steps_per_epoch = train_samples//batch_size,
#                     epochs=epochs,
#                     validation_data = generate_batch(X_test, y_test, batch_size = batch_size),
#                     validation_steps = val_samples//batch_size)

history = model.fit(generate_batch(X_train, y_train, batch_size=batch_size),
                    steps_per_epoch=train_samples // batch_size,
                    epochs=epochs,
                    validation_data=generate_batch(X_test, y_test, batch_size=batch_size),
                    validation_steps=val_samples // batch_size,
                     callbacks=[model_checkpoint_callback])


Epoch 1/10
  3/238 [..............................] - ETA: 7:02 - loss: 2.0706 - accuracy: 0.6197

KeyboardInterrupt: 

In [ ]:
# model.save_weights(r'/content/drive/MyDrive/Honours-DL/Translation/Translation-Transformer-1-MR-EN.h5')

In [ ]:
model.load_weights(r'/content/drive/MyDrive/Honours-DL/Translation/temp.h5')

In [ ]:
import tensorflow as tf
model = tf.keras.models.load_model(r'/content/drive/MyDrive/Honours-DL/Translation/Translation-Transformer-1-MR-EN-2.h5')

In [ ]:
model.save(r'/content/drive/MyDrive/Honours-DL/Translation/Translation-Transformer-1-MR-EN-complete-sub-word.h5')

In [ ]:
import tensorflow as tf
model = tf.keras.models.load_model(checkpoint_filepath)

In [ ]:
# Encode the input sequence to get the "thought vectors"
encoder_model = Model(encoder_inputs, encoder_states)

# Decoder setup
# Below tensors will hold the states of the previous time step
decoder_state_input_h = Input(shape=(latent_dim,))
decoder_state_input_c = Input(shape=(latent_dim,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]

dec_emb2= dec_emb_layer(decoder_inputs) # Get the embeddings of the decoder sequence
# To predict the next word in the sequence, set the initial states to the states from the previous time step
decoder_outputs2, state_h2, state_c2 = decoder_lstm(dec_emb2, initial_state=decoder_states_inputs)
decoder_states2 = [state_h2, state_c2]
decoder_outputs2 = decoder_dense(decoder_outputs2) # A dense softmax layer to generate prob dist. over the target vocabulary

# Final decoder model
decoder_model = Model(
    [decoder_inputs] + decoder_states_inputs,
    [decoder_outputs2] + decoder_states2)

ValueError: Exception encountered when calling layer "lstm_5" (type LSTM).

Dimensions must be equal, but are 128 and 256 for '{{node MatMul_1}} = MatMul[T=DT_FLOAT, transpose_a=false, transpose_b=false](init_h, recurrent_kernel)' with input shapes: [?,128], [256,1024].

Call arguments received by layer "lstm_5" (type LSTM):
  • inputs=['tf.Tensor(shape=(None, None, 128), dtype=float32)', 'tf.Tensor(shape=(None, 128), dtype=float32)', 'tf.Tensor(shape=(None, 128), dtype=float32)']
  • mask=['tf.Tensor(shape=(None, None), dtype=bool)', 'None', 'None']
  • training=None
  • initial_state=None

In [ ]:
encoder_model = Model(encoder_inputs, encoder_states)

decoder_state_input_h = Input(shape=(latent_dim*2,))
decoder_state_input_c = Input(shape=(latent_dim*2,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
print(decoder_states_inputs)

# Use the same embedding layer initialized previously for subword tokenization
dec_emb2 = dec_emb_layer(decoder_inputs)

decoder_outputs2, state_h2, state_c2 = decoder_lstm(dec_emb2, initial_state=decoder_states_inputs)
decoder_states2 = [state_h2, state_c2]
decoder_outputs2 = decoder_dense(decoder_outputs2)  # A dense softmax layer to generate prob dist. over the target vocabulary

decoder_model = Model(
    [decoder_inputs] + decoder_states_inputs,
    [decoder_outputs2] + decoder_states2)


[<KerasTensor: shape=(None, 256) dtype=float32 (created by layer 'input_21')>, <KerasTensor: shape=(None, 256) dtype=float32 (created by layer 'input_22')>]


In [ ]:
def decode_sequence(input_seq):
    # Encode the input as state vectors.
    states_value = encoder_model.predict(input_seq)

    # return len(states_value), len(states_value[0]), len(states_value[1])
    # Generate empty target sequence of length 1.
    target_seq = np.zeros((1,1))

    # Populate the first character of target sequence with the start character.
    target_seq[0, 0] = target_token_index['START_']

    # Sampling loop for a batch of sequences
    # (to simplify, here we assume a batch of size 1).
    stop_condition = False
    decoded_sentence = ''

    while not stop_condition:
        output_tokens, h, c = decoder_model.predict([target_seq] + states_value)

        # Sample a token
        sampled_token_index = np.argmax(output_tokens[0, -1, :])

        sampled_char = reverse_target_char_index[sampled_token_index]
        decoded_sentence += ' '+sampled_char

        # Exit condition: either hit max length
        # or find stop character.
        if (sampled_char == '_END' or
           len(decoded_sentence) > 100):
            stop_condition = True

        # Update the target sequence (of length 1).
        target_seq = np.zeros((1,1))
        target_seq[0, 0] = sampled_token_index

        # Update states
        states_value = [h, c]

    return decoded_sentence

In [ ]:
def decode_sequence(input_seq):
    # Encode the input as state vectors.
    states_value = encoder_model.predict(input_seq)
    # Generate empty target sequence of length 1.
    target_seq = np.zeros((1,1))
    # Populate the first character of target sequence with the start character.
    target_seq[0, 0] = target_token_index['START_']

    # Sampling loop for a batch of sequences
    # (to simplify, here we assume a batch of size 1).
    stop_condition = False
    decoded_sentence = ''
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict([target_seq] + states_value)

        # Sample a token
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = reverse_target_char_index[sampled_token_index]
        decoded_sentence += ' '+sampled_char

        # Exit condition: either hit max length
        # or find stop character.
        if (sampled_char == '_END' or
           len(decoded_sentence) > 100):
            stop_condition = True

        # Update the target sequence (of length 1).
        target_seq = np.zeros((1,1))
        target_seq[0, 0] = sampled_token_index

        # Update states
        states_value = [h, c]

    return decoded_sentence

In [ ]:
def decode_sequence(input_seq):
    # Encode the input as state vectors.
    states_value = encoder_model.predict(input_seq)

    # Generate empty target sequence of length 1.
    target_seq = np.zeros((1, 1))

    # Populate the first subword token of target sequence with the start token.
    target_seq[0, 0] = target_token_index['START_']

    # Initialize an empty decoded sentence
    decoded_sentence = ''

    # Sampling loop for a sequence
    stop_condition = False
    while not stop_condition:
        # Predict the next subword token
        output_tokens, h, c = decoder_model.predict([target_seq] + states_value)

        # Sample a token index
        sampled_token_index = np.argmax(output_tokens[0, -1, :])

        # Map the sampled token index to the subword token
        sampled_subword = reverse_target_char_index[sampled_token_index]

        # Append the subword to the decoded sentence
        decoded_sentence += sampled_subword

        # Exit condition: either hit max length or find stop token
        if (sampled_subword == '_END' or len(decoded_sentence) > 100):
            stop_condition = True

        # Update the target sequence to contain the sampled token for the next iteration
        target_seq = np.zeros((1, 1))
        target_seq[0, 0] = sampled_token_index

        # Update states
        states_value = [h, c]

    return decoded_sentence


In [ ]:
train_gen = generate_batch(X_train, y_train, batch_size = 1)
k=-1

In [ ]:
k+=1
(input_seq, actual_output), _ = next(train_gen)
print(input_seq)
decoded_sentence = decode_sequence(input_seq)
print('Input Marathi sentence:', X_train[k:k+1].values[0])
print('Actual English Translation:', y_train[k:k+1].values[0][6:-4])
print('Predicted English Translation:', decoded_sentence)

[[27425. 31179. 37389.  4938.  3197. 14536. 26699. 12611.     0.     0.
      0.     0.     0.     0.     0.     0.     0.     0.     0.     0.]]
1/1 [==============================] - 0s 26ms/step
Input Marathi sentence: या विकृत हत्येतील एका आरोपीचा तुरुंगात मृत्यू झाला
Actual English Translation:  one accused died in jail 
Predicted English Translation:  one of people have been arrested _END
